# Pipeline : 
### data processing of the manually annoted dataset
### image processing : extraction of RGB volumes
### classifier based on WND-charm Model
### PCA & RDF
### Build a classifier
### Analyse False Positives
### Prediction of Neurons, Astrocytes, Background in ROI's clone

In [1]:
import numpy as np
import pandas as pd
from functools import partial
import multiprocessing
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.colors as cm
import pickle
import time
from pandas import DataFrame, read_csv,concat,merge

In [2]:
%run ./configure.ipynb

In [3]:
%run ../Connection_to_server.ipynb

In [4]:
imageId = 451
weight=25
height=25

## Pre-pocessing of the manually annotated Dataset

In [5]:
import Pre_Processing_the_annotated_dataset as pp
reload(pp);

In [6]:
dfpoints1=read_csv(projectpath+'io/Input/Bloc01_pointes_first_mod.csv')


In [7]:
# Bloc 2
dfpoints2=read_csv(projectpath+'io/Input/Bloc02_pointes_second.csv')

In [8]:
import Pre_Classification_process as pcp
reload(pcp);

In [9]:
filename_Features_bloc1=projectpath+'io/Output/Features_bloc1.npy'

In [10]:
Features_bloc1=np.load(filename_Features_bloc1)

In [11]:
Data_FRAMES_bloc1=pd.read_pickle(projectpath+'io/Output/Data_FRAMES_bloc1.pkl')

In [14]:
filename_Features_bloc2=projectpath+'io/Output/Features_bloc2.npy'

In [15]:
Features_bloc2=np.load(filename_Features_bloc2)

In [16]:
Data_FRAMES_bloc2=pd.read_pickle(projectpath+'io/Output/Data_FRAMES_bloc2.pkl')

# Create the full data Frame ( 1135 Adtrocytes, 1135 Neurons, 1151 background) 

In [17]:
import join_two_blocs as jt
reload(jt);
import Fonctions_used_for_many_DF_pre_processing as fct_pp
reload(fct_pp);

In [18]:
Data_FRAMES=pd.read_pickle(projectpath+'io/Output/Dataframe_.pkl')

In [19]:
filename_Features_two_blocs=projectpath+'io/Output/Features_two_blocs.npy'

In [20]:
Features_two_blocs=np.load(filename_Features_two_blocs)

# PCA & Random Forest Classification

In [21]:
%%time
%run ./pca_rdf_models.ipynb
#%run ./Pipeline_PCA_RANDOM_Forest_Copy1.ipynb
# CPU times: user 16min 53s, sys: 1h 52s, total: 1h 17min 45s
# Wall time: 6min 58s
#CPU times: user 2h 17min 17s, sys: 2h 13min 33s, total: 4h 30min 50s
#Wall time: 25min 13s

/home/tbacoyannis/.local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/tbacoyannis/.local/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Pipeline(steps=[('reduce_dim', PCA(copy=True, iterated_power='auto', n_components=80, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('Random_Forest', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=...stimators=200, n_jobs=1, oob_score=False, random_state=0,
            verbose=0, warm_start=False))])
CPU times: user 18min 39s, sys: 1h 1min 1s, total: 1h 19min 41s
Wall time: 7min 51s


# Imshow specific images to understand non differentiation in PCA model

# process in order to display false predicted images

In [ ]:
(fail_img_pred_Neurons,fail_img_pred_Astrocytes,fail_img_pred_Background)=imf.Get_IMG_failed_prediction(yr,predictedVAL)

In [ ]:
sns.reset_orig() # sns.set()

# test prediction sur  1 clone

In [22]:
import Get_coordinates as gC
reload(gC);

In [23]:
conn = BlitzGateway('tbacoyannis','d33pl34rn1ng',port=4064,host='chinensis.polytechnique.fr')
connected = conn.connect()

image = conn.getObject("Image", imageId)
pixels = image.getPrimaryPixels() # get raw pixels information

size_x=image.getSizeX()
size_y=image.getSizeY()
size_z = image.getSizeZ()
size_c = image.getSizeC()
size_t = image.getSizeT()

large= weight  
nidY=size_x/(int(large/int(large/2)))-3
delta_x=2
z=103

In [24]:
# Coordinates of the ROI in the original image
x_min=48+367
y_min=2064+646
w=615-367
h=920-646
x_max=x_min+w
y_max=y_min+h

In [25]:
conn = BlitzGateway('tbacoyannis','d33pl34rn1ng',port=4064,host='chinensis.polytechnique.fr')
connected = conn.connect()
imageId=451
image = conn.getObject("Image", imageId)
pixels = image.getPrimaryPixels() # get raw pixels information

pxobj_CH1 = pixels.getTile(theZ=103, theC=0, theT=0, tile=(x_min,y_min, x_max-x_min , y_max-y_min ))
pxobj_CH2 = pixels.getTile(theZ=103, theC=1, theT=0, tile=(x_min,y_min, x_max-x_min , y_max-y_min ))
pxobj_CH3 = pixels.getTile(theZ=103, theC=2, theT=0, tile=(x_min,y_min, x_max-x_min , y_max-y_min ))

# Get the normalized original image
pxobj=(pxobj_CH1+pxobj_CH2+pxobj_CH3)/3

In [26]:
partial_getDATA_coord = partial(gC.get_coordinates_test,imageId=imageId, large=large,delta_x= delta_x,z=z,x_min=x_min,y_min=y_min,x_max=x_max, y_max=y_max)

In [27]:
filename_resultat =projectpath+'io/Output/resultat.npy'

In [28]:
res=np.load(filename_resultat)
M_Mv=res[:,0]
liste_coordonnees=res[:,1]


In [29]:
# Reshape the list of coordinates
b=np.vstack(liste_coordonnees)

x_from_liste_coordonnees=b[:,0]
y_from_liste_coordonnees=b[:,1]
ind_lign_from_liste_coordonnees=b[:,2]
ind_col_from_liste_coordonnees=b[:,3]

In [30]:
Matrix_TEST=np.vstack(M_Mv)
Matrix_TEST=Matrix_TEST/255.0

In [31]:
coords=np.where((np.hstack(x_from_liste_coordonnees)>=x_min) & (np.hstack(x_from_liste_coordonnees)<=x_max) & (np.hstack(y_from_liste_coordonnees)>=y_min) & (np.hstack(y_from_liste_coordonnees)<=y_max))[0]

In [32]:
Im_to_convert_GRAY=Matrix_TEST  #[coords]

In [33]:
Im_to_convert_GRAY.shape

(30752, 25, 25, 3)

In [34]:
filename_Im_to_convert_GRAY=projectpath+'io/Output/Im_to_convert_GRAY.npy'

In [35]:
np.save(filename_Im_to_convert_GRAY,Im_to_convert_GRAY)

In [36]:
Im_to_convert_GRAY=np.load(filename_Im_to_convert_GRAY)

In [37]:
Im_to_convert_GRAY.shape

(30752, 25, 25, 3)

## Process for one tile ipynb

In [80]:
import process_for_one_tile as pt
reload(pt);

In [81]:
nb_features=2919

In [82]:
size_=Im_to_convert_GRAY.shape[0]-1
nidx=Im_to_convert_GRAY.shape[0]

In [83]:
partial_Get_feat_test = partial(pt.Get_feat_test, h=height, w=weight, M_mean=Im_to_convert_GRAY, nb_features=nb_features)

In [96]:
filename_FV_test =projectpath+'io/Output/FV_test.npy'

In [97]:
FV_test=np.load(filename_FV_test)

In [102]:
from sklearn.externals import joblib

In [103]:
pca_FV_test=clf_pca.transform(FV_test)

In [106]:
prediction_TEST=clf_RFC.predict(FV_test)

In [93]:
filename_prediction_TEST =projectpath+'io/Output/prediction_TEST.npy'
np.save(filename_prediction_TEST,prediction_TEST)

In [94]:
prediction_TEST=np.load(filename_prediction_TEST)

In [62]:
predict_probability=clf_RFC.predict_proba(FV_test)

In [73]:
filename_predict_probability=projectpath+'io/Output/predict_probability.npy'
np.save(filename_predict_probability,predict_probability)

In [74]:
predict_probability=np.load(filename_predict_probability)

In [95]:
prediction_TEST

array([ 2.,  2.,  2., ...,  2.,  2.,  2.])

In [ ]:
################################"

In [ ]:
Im_proba=np.zeros([Im_Proba.shape[0], Im_Proba.shape[2]])

for g in range(0,Im_Proba.shape[0]):
    Im_proba[g]=Im_Proba[g][g]

In [ ]:
filename_Im_proba =projectpath+'io/Output/Im_proba.npy'

In [ ]:
Im_proba=np.save(filename_Im_proba,Im_proba)

In [ ]:
Im_proba=np.load(filename_Im_proba)

In [ ]:
Probability_img=np.zeros([Im_proba.shape[0]+1,Im_proba.shape[1]])
Probability_img[:Im_proba.shape[0]]=Im_proba
Probability_img[Im_proba.shape[0]]=Probability_img[Im_proba.shape[0]-1]

In [ ]:
conn = BlitzGateway('tbacoyannis','d33pl34rn1ng',port=4064,host='chinensis.polytechnique.fr')
connected = conn.connect()
imageId=451
image = conn.getObject("Image", imageId)
pixels = image.getPrimaryPixels() # get raw pixels information

pxobj_CH1 = pixels.getTile(theZ=103, theC=0, theT=0, tile=(x_min,y_min, x_max-x_min , y_max-y_min ))
pxobj_CH2 = pixels.getTile(theZ=103, theC=1, theT=0, tile=(x_min,y_min, x_max-x_min , y_max-y_min ))
pxobj_CH3 = pixels.getTile(theZ=103, theC=2, theT=0, tile=(x_min,y_min, x_max-x_min , y_max-y_min ))

# Get the normalized original image
pxobj=(pxobj_CH1+pxobj_CH2+pxobj_CH3)/3

In [ ]:
e=124
CH1=Probability_img[:,0]
CH1=np.reshape(CH1, (-1, e) )
CH1=CH1[:,:, np.newaxis]
#     plt.title('Proba Astrocytes')

CH2=Probability_img[:,1]
CH2=np.reshape(CH2,(-1,e))
CH2=CH2[:,:,np.newaxis]
#     plt.title('Proba Neurons')


CH3=Probability_img[:,2]
CH3=np.reshape(CH3, (-1, e) )
CH3=CH3[:,:,np.newaxis]
#     plt.title('Proba Background')

Z=np.dstack([CH1,CH2,CH3])

In [ ]:
filename_RGB_proba_im =projectpath+'io/Output/RGB_proba_im.npy'

In [ ]:
np.save(filename_RGB_proba_im,Z)

In [ ]:
Z=np.load(filename_RGB_proba_im)

In [ ]:
import IM_Display_RGB as imd
reload(imd);

In [ ]:
imd.imshowRGB(Z,pxobj,10)